In [8]:
#IMPORT DEPENDENCIES
import sys; sys.path.insert(0, 'lib')
import os
import copy
import json
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import date

#relative imports of Isotomics
cwd = Path().resolve()
print(cwd)
sys.path.insert(1, os.path.join(cwd, 'lib'))
import fragmentAndSimulate as fas
import calcIsotopologues as ci
import readInput as ri
import solveSystem as ss
import basicDeltaOperations as op
import fragmentAndSimulate as fas 
import organizeData
import dataAnalyzerMNIsoX as dA
import dataScreenIsoX as dataScreen
import readCSVAndSimulate as sim
import spectrumVis

/Users/sarahzeichner/Documents/GitHub/Isotomics-Automated


In [9]:
#SET PARAMETERS
MOLECULE_INPUT_PATH = "Processed Data\Example Input.csv"
STD_DELTAS = [-30] * 6
SMP_DELTAS = [0] * 6
STD_DELTAS_APPROX = [-30] * 6
ABUNDANCE_THRESHOLD = 0
OMIT_MEASUREMENTS = {'M1':{'full':['17O']}}
FRACTIONATION = False
FRACTIONATION_AMOUNT = 0.05
ERROR_PATH = 'Processed Data\Alanine Experimental Errors.csv'
MN_ERROR = 1
MOLECULAR_AVERAGE_SUBSTITUION = '13C'
MOLECULAR_AVERAGE_ERROR = 1
OUTPUT_FILENAME = 'output.csv'

In [10]:
#CONSTRUCT AND EXPLORE SYSTEM
initializedMolecule = sim.moleculeFromCsv(MOLECULE_INPUT_PATH)
predictedMeasurement, MNDict, fractionationFactors = sim.simulateMeasurement(initializedMolecule, massThreshold = 5)

spectrumVis.fullSpectrumVis(predictedMeasurement['Full Molecule'], initializedMolecule['molecularDataFrame'], figsize = (8,4), massError = 0, lowAbundanceCutOff = 0, xlim = (), ylim = (), outputIsotopologs=True)

spectrumVis.MNSpectrumVis(initializedMolecule['molecularDataFrame'], '44', predictedMeasurement, 'M1', MNDict, lowAbundanceCutOff = 0, massError = 0, xlim = (), ylim = (), outputIsotopologs=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Processed Data\\Example Input.csv'

In [11]:
#GET SIMULATION OUTPUT
simulationOutput = sim.simulateSmpStd(MOLECULE_INPUT_PATH, STD_DELTAS, SMP_DELTAS, STD_DELTAS_APPROX, abundanceThreshold = 0, UValueList = [MOLECULAR_AVERAGE_SUBSTITUION], calcFF = FRACTIONATION, omitMeasurements = OMIT_MEASUREMENTS, ffstd = FRACTIONATION_AMOUNT, errorPath = ERROR_PATH, MNError = MN_ERROR, UValueError = MOLECULAR_AVERAGE_ERROR, resultsFileName = OUTPUT_FILENAME)

FileNotFoundError: [Errno 2] No such file or directory: 'Processed Data\\Example Input.csv'

In [12]:
#SPECIFY DETAILS ABOUT THE MEASUREMENT
today = date.today()

#Constants for the runs for processing
ACQUISITION_LENGTH = (5, 15) #Change the time here based on how long it takes isotope val to stabilize.
STANDARD_VAL = -31.13 #d13C value of VAN4
NUMBER_OF_CARBONS = 3
STANDARD_DELTA_VALS = [-20.19,-27.7,-62.47,0,0,0,0,0] #initiate array with values for VANA, or whatever is the standard

In [13]:
#CREATE FOLDERS FOR DATA PROCESSING
#set variables related to where data will be output to
output_folder_name = 'Test_Data_Output' #name your folder that you want all your data to go into (e.g., the name of your experiment)
processed_data_subfolder_name = 'Processed Data'
file_extension = '.isox' #set the extension to be whatever format your files are in. currently, the code handles .isox files

#generate nested folders to put your data into
fragmentDictionary = initializedMolecule("/Users/sarahzeichner/Documents/GitHub/Isotomics-User-Friendly/Alanine Tutorial/Example Input.csv")['fragmentationDictionary'] 
new_folder_path = os.path.join(cwd, processed_data_subfolder_name, output_folder_name)
folderNames, parent_folder = organizeData.create_nested_folders(fragment_dict=fragmentDictionary.keys(), parent_folder=new_folder_path)

NameError: name 'initializedMolecule' is not defined

In [ ]:
#PROCESS DATA FROM FOLDER PATHS
fragmentFolderPaths = organizeData.get_subfolder_paths(parent_folder)
print(fragmentFolderPaths)

#TODO: CREATE A LOOP WHERE ALL THESE DIFFERENT FOLDERS GET ITERATED THROUGH AND THEN SPIT OUT DATA 
#Get list of file names and paths to process
isoXFileNames = organizeData.get_isoX_file_Smp_Std_Dictionary(parent_folder, file_extension)

for thisFolder in isoXFileNames.items():
    dA.calc_Folder_Output(thisFolder)
print(isoXFileNames)